In [12]:
using GAP
using JSON3


In [13]:
# json ファイルの読み取り
config_raw = JSON3.read(read("config.json", String))

"""
JSONのサイクルリストからGAPの置換オブジェクトを生成する
例: [[1,3,8,6], [2,5,7,4]]  =>  (1,3,8,6)(2,5,7,4)
"""
function reconstruct_gap_perm(cycles_list)
    # 各サイクルを "(1,3,8,6)" のような文字列に変換し、結合する
    # GAP.evalstr は文字列をGAPの式として評価してオブジェクトを返します
    gap_str = join(["(" * join(c, ",") * ")" for c in cycles_list], "")

    # 恒等置換（どこも動かない）の場合は空文字列になるので、その場合は "()" を渡す
    if gap_str == ""
        return @gap ()
    end

    return GAP.evalstr(gap_str)
end


reconstruct_gap_perm

In [14]:
# 2. 各面を再構築して辞書に格納
# これで Core 定義を外部ファイルから注入できます
moves = Dict()
for (face_name, cycles) in config_raw.definitions
    moves[string(face_name)] = reconstruct_gap_perm(cycles)
end


In [15]:
# 3. 以前の Core 定義のように変数へ割り当てる
U, L, F, R, B, D = moves["U"], moves["L"], moves["F"], moves["R"], moves["B"], moves["D"]

cube = (@gap Group)(U, L, F, R, B, D)
words = @gap ["U", "L", "F", "R", "B", "D"]
free = (@gap FreeGroup)(words)
hom = (@gap GroupHomomorphismByImages)(free, cube, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube))

scramble_array = [rand(["U", "L", "F", "R", "B", "D"]) for _ in 1:20]
scramble_str = join(scramble_array, " ")

σ = @gap ()
for s in scramble_array
    idx = findfirst(==(s), ["U", "L", "F", "R", "B", "D"])
    global σ *= (@gap GeneratorsOfGroup)(cube)[idx]
end

word_raw = string((@gap PreImagesRepresentative)(hom, σ))

function gap_to_lsystem(word_str)
    res = replace(word_str, "*" => " ")
    res = replace(res, "^-1" => "-")
    res = replace(res, "^-2" => "++")
    res = replace(res, "^2" => "++")
    return join([occursin(r"[\+\-]", m) ? m : m * "+" for m in split(res)], " ")
end


gap_to_lsystem (generic function with 1 method)

In [26]:
"""
現在のステッカー配置（current_state）に、
指定された置換（p_gap: GAPのオブジェクト）を適用して新しい配置を返す
"""
function apply_move_to_state(current_state::Vector{Int}, p_gap)
    # 1. Juliaのループで各番号の移動先を計算する
    # GAP.jlにより、整数 ^ GapObj (置換) で移動後の値が返ります
    p_map = [Int(i^p_gap) for i in 1:48]
    
    # 2. 新しい状態を作る（物理的な移動）
    next_state = copy(current_state)
    for i in 1:48
        next_state[p_map[i]] = current_state[i]
    end
    
    return next_state
end

# 動作確認
current_state = collect(1:48)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, U)
println(current_state)


[1, 2, 3, 4, 5, 16, 13, 11, 9, 10, 41, 12, 42, 14, 15, 43, 22, 20, 17, 23, 18, 24, 21, 19, 6, 26, 27, 7, 29, 8, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 30, 28, 25, 44, 45, 46, 47, 48]


In [ ]:
data = Dict(
    "current" => current_state,
    "scramble" => gap_to_lsystem(scramble_str),
    "solution" => gap_to_lsystem(word_raw)
)
open("../src/config/solve.json", "w") do f
    JSON3.pretty(f, data)
end


In [10]:
using GAP

operators = Dict(
    "U" => @gap((1, 3, 8, 6) * (2, 5, 7, 4) * (9,33,25,17) * (10,34,26,18) * (11,35,27,19)),
    "L" => @gap((9,11,16,14) * (10,13,15,12) * (1,17,41,40) * (4,20,44,37) * (6,22,46,35)),
    "F" => @gap((17,19,24,22) * (18,21,23,20) * (6,25,43,16) * (7,28,42,13) * (8,30,41,11)),
    "R" => @gap((25,27,32,30) * (26,29,31,28) * (3,38,43,19) * (5,36,45,21) * (8,33,48,24)),
    "B" => @gap((33,35,40,38) * (34,37,39,36) * (3, 9,46,32) * (2,12,47,29) * (1,14,48,27)),
    "D" => @gap((41,43,48,46) * (42,45,47,44) * (14,22,30,38) * (15,23,31,39) * (16,24,32,40))
)

# --- 2. 初期状態の設定 ---
state = collect(1:48)         
history = [copy(state)]       
current_perm = @gap(())       # 単位元は空のカッコでOK

move_pool = ["U", "L", "F", "R", "B", "D", "U'", "L'", "F'", "R'", "B'", "D'"]

for i in 1:10
    m = rand(move_pool)
    println("Step $i: Applying move $m")
    
    is_inverse = endswith(m, "'") || endswith(m, "-")
    base_move = is_inverse ? m[1:end-1] : m
    
    p = get(operators, base_move, @gap(()))
    actual_perm = is_inverse ? inv(p) : p
    
    p_map = [Int(j^actual_perm) for j in 1:48]
    
    next_state = copy(state)
    for j in 1:48
        next_state[p_map[j]] = state[j]
    end
    
    state = next_state
    current_perm *= actual_perm
    push!(history, copy(state))
end

gens = [operators["U"], operators["L"], operators["F"], operators["R"], operators["B"], operators["D"]]
gen_names = ["U", "L", "F", "R", "B", "D"]

# 2. 生成元から「群」を定義する
free_group = GAP.Globals.FreeGroup(length(gen_names))
phi = GAP.Globals.EpimorphismFromFreeGroup(free_group, cube_group)

# 4. 解法の計算
target = inv(current_perm)

println("\nCalculating solution...")
# PreImagesRepresentative を使い、target を生成元の積として表現する
# ※非常にバラバラな場合は計算に時間がかかることがありますが、10手程度なら一瞬です
word = GAP.Globals.PreImagesRepresentative(phi, target)

solution_str = GAP.Globals.String(word)
println("Solution: ", solution_str)


Step 1: Applying move L'
Step 2: Applying move R'
Step 3: Applying move D'
Step 4: Applying move R'
Step 5: Applying move U
Step 6: Applying move D'
Step 7: Applying move B'
Step 8: Applying move F
Step 9: Applying move L'
Step 10: Applying move B


LoadError: UndefVarError: `julia_to_gap` not defined in `GAP`
Suggestion: check for spelling errors or missing imports.

In [20]:
using GAP
using JSON3

# --- 1. 定義・初期化系（純粋関数） ---

"""JSONのサイクルからGAPの置換を作る"""
function reconstruct_gap_perm(cycles)
    gap_str = join(["(" * join(c, ",") * ")" for c in cycles], "")
    return gap_str == "" ? (@gap ()) : GAP.evalstr(gap_str)
end

"""GAPの準同型写像（ソルバーの核）を作成する"""
function create_homomorphism(operators)
    names_jl = ["U", "L", "F", "R", "B", "D"]
    gen_list = [operators[n] for n in names_jl]
    
    names_gap = @gap ["U", "L", "F", "R", "B", "D"]
    cube_group = (@gap Group)(gen_list...)
    free_group = (@gap FreeGroup)(names_gap)
    return (@gap GroupHomomorphismByImages)(
        free_group, 
        cube_group, 
        (@gap GeneratorsOfGroup)(free_group), 
        (@gap GeneratorsOfGroup)(cube_group)
    )
end

# --- 2. 状態管理（構造体） ---

mutable struct RubikCube
    state::Vector{Int}
    operators::Dict{String, Any}
    hom::Any
    current_perm::Any
    scramble::String # メタデータとして保持

    function RubikCube(config_path::String, solve_json_path::String)
        config = JSON3.read(read(config_path, String))
        solve_data = JSON3.read(read(solve_json_path, String))
        
        ops = Dict(String(k) => reconstruct_gap_perm(v) for (k, v) in config.definitions)
        hom = create_homomorphism(ops)
        
        new(Vector{Int}(solve_data.current), ops, hom, (@gap ()), solve_data.scramble)
    end
end
# --- 3. 操作・計算メソッド ---

"""手順を適用して状態と累積置換を更新する"""
function apply_moves!(cube::RubikCube, move_names::Vector{String})
    history = Vector{Vector{Int}}()
    for m in move_names
        # 末尾に ' か - があれば逆回転とみなす
        is_inverse = endswith(m, "'") || endswith(m, "-")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, (@gap ()))
        actual_perm = is_inverse ? inv(perm) : perm
        
        p_map = [Int(i^actual_perm) for i in 1:48]
        next_state = copy(cube.state)
        for i in 1:48
            next_state[p_map[i]] = cube.state[i]
        end
        cube.state = next_state
        cube.current_perm *= actual_perm
        push!(history, copy(cube.state))
    end
    return history
end

function find_solution(cube::RubikCube)
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    # 計算前に GAP 側で強連結な生成元を再整理させる (StabChain の最適化)
    # (@gap MakeStabChain)(cube.cube_group) 
    
    word = (@gap PreImagesRepresentative)(cube.hom, inv(g))
    
    # Julia 側でキャストして最適化へ回す
    raw_str = String((@gap String)(word))
    
    # 1. 粗い展開
    expanded_moves = split(gap_to_lsystem(raw_str))
    
    # 2. 物理的な圧縮
    optimized_moves = optimize_moves(Vector{String}(expanded_moves))
    
    return join(optimized_moves, " ")
end


# --- 1. 準同型写像の準備 (以前の定義を使用) ---
hom = create_homomorphism(operators)

# --- 2. 状態の読み込みと変換 ---
target_state = [
    38, 21, 22, 4, 20, 3, 44, 11, 48, 10, 27, 23, 39, 25, 5, 43, 33, 15, 17, 47, 
    34, 24, 18, 1, 6, 13, 16, 2, 29, 9, 45, 40, 41, 28, 32, 36, 42, 14, 37, 8, 
    30, 7, 35, 26, 31, 19, 12, 46
]

# evalstr を使って GAP のリストを作り、置換オブジェクトを生成
gap_list = GAP.evalstr("[" * join(target_state, ",") * "]")
g = (@gap PermList)(gap_list)

# --- 3. 解法の計算 ---
target_inv = inv(g)

println("Calculating solution...")
word = (@gap PreImagesRepresentative)(hom, target_inv)

# 重要：(@gap String) の結果を Julia の String にキャストする
solution_raw_gap = (@gap String)(word)
solution_raw_jl = String(solution_raw_gap) # ここで GapObj から String へ変換

# 文字列置換処理 (これでエラーが出なくなります)
final_solution = gap_to_lsystem(solution_raw_jl)

println("\n--- Solution Found ---")
println(final_solution)


Calculating solution...

--- Solution Found ---
B R- U F R- U- B- R B U- D- B- D U F F U U F- L U- F U F L F- L- F- U- L L B- U B U U L- F- L- F L U- F U F- U- L F- L- F L- U- L U U F- L- U- L U F U- L U L- U L U- U- L- L- U- L U L F- L- F F U F- U- F- L F L L B- U- B U L U F U R U- R- F- U-


In [51]:
using GAP

# --- 1. 定義セクション ---

mutable struct RubikCube
    state::Vector{Int}
    operators::Dict{String, Any}
    hom::Any
end

"""手順を適用して状態を更新する"""
function apply_moves!(cube::RubikCube, move_names::Vector{String})
    for m in move_names
        m = strip(m)
        if isempty(m) continue end
        
        # 逆回転判定: 末尾が '-' の場合
        is_inverse = endswith(m, "-")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, nothing)
        if perm === nothing
            # @warn "無効な操作をスキップ: $m"
            continue
        end
        
        actual_perm = is_inverse ? inv(perm) : perm
        
        # ステッカーの移動 (場所 i のステッカーを i^P へ飛ばす)
        p_map = [Int(i^actual_perm) for i in 1:48]
        next_state = copy(cube.state)
        for i in 1:48
            next_state[p_map[i]] = cube.state[i]
        end
        cube.state = next_state
    end
end

"""
GAP の複雑な表記 (U^3, R^-1, (U*L)^2 等) を一手ずつの配列に分解する
正規表現を用いた堅牢なパースロジック
"""
function gap_to_lsystem(word_str)
    # 不要な記号を削除
    res = replace(word_str, "*" => " ", "(" => "", ")" => "")
    
    final_moves = String[]
    for p in split(res)
        # 正規表現で面(A-Z)と指数部分を抽出
        # U^3 -> 面:U, 指数:3 | R^-1 -> 面:R, 指数:-1 | F -> 面:F, 指数:nothing
        m = match(r"([A-Z])(?:\^?([\-\d]+))?", p)
        if m === nothing continue end
        
        face = m.captures[1]
        pow_str = m.captures[2]
        
        # 指数（回転数）を整数化。デフォルトは1（時計回り90度）
        pow = pow_str === nothing ? 1 : parse(Int, pow_str)
        
        # 物理的な回転数に正規化 (4回で一周)
        pow = pow % 4
        if pow < 0 pow += 4 end
        
        if pow == 1       # 90度
            push!(final_moves, face)
        elseif pow == 2   # 180度 (2回分に展開)
            push!(final_moves, face); push!(final_moves, face)
        elseif pow == 3   # 270度 (反時計回り90度として扱う)
            push!(final_moves, face * "-")
        end
    end
    return final_moves # 配列として返す
end

"""現在の状態 state[i] = S から解法を導く"""
function find_solution(cube::RubikCube)
    # 数学的な正解:
    # 現在の場所 i にあるステッカー ID が state[i] であるとき、
    # その状態を初期状態に戻すための置換は PermList(state) そのものである。
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    # ターゲット g をそのまま解けば、正順で適用できる手順が得られる
    word = (@gap PreImagesRepresentative)(cube.hom, g)
    return gap_to_lsystem(String((@gap String)(word)))
end

# --- 2. 実行セクション ---

# オペレータ定義 (変更なし)
operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

# 準同型写像作成用ヘルパー
function setup_hom(ops)
    names = ["U", "L", "F", "R", "B", "D"]
    names_gap = GAP.evalstr("[\"U\", \"L\", \"F\", \"R\", \"B\", \"D\"]")
    free = (@gap FreeGroup)(names_gap)
    cube_grp = (@gap Group)([ops[n] for n in names]...)
    return (@gap GroupHomomorphismByImages)(free, cube_grp, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube_grp))
end

# インスタンス生成
cube = RubikCube(collect(1:48), operators, setup_hom(operators))

println("Scrambling...")
# ランダムに崩す
scramble_input = ["U", "L-", "F", "R", "D", "B-"]
apply_moves!(cube, scramble_input)

println("\nSolving...")
sol_moves = find_solution(cube) # 配列が返ってくる
println("Solution Sequence: ", join(sol_moves, " "))

println("\nApplying Solution...")
apply_moves!(cube, sol_moves)

println("\n--- Result ---")
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Check parsing or operators.")
    # 差分を表示
    println("Mismatched indices: ", findall(i -> cube.state[i] != i, 1:48))
end


Scrambling...

Solving...
Solution Sequence: D- B- R- B L- D- B D L F- L B- U- B U- L- F- L F U L F- L F F U F- U- L- U L- U- B L- B- L U L L F U F- U- L- U L F U F- L- U- L- U B L B- U U F U F- L- U- L U F L- U- L L F- L- F F U- F- L U F U- F- L-

Applying Solution...

--- Result ---
FAILED: Check parsing or operators.
Mismatched indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 33, 34, 35, 41, 42, 43]


In [70]:
using GAP

# --- 1. 定義セクション ---

mutable struct RubikCube
    state::Vector{Int}
    operators::Dict{String, Any}
    hom::Any
end

"""手順を適用して状態ベクトルを更新する"""
function apply_moves!(cube::RubikCube, move_names::Vector{String})
    for m in move_names
        m = strip(m)
        if isempty(m) continue end
        is_inverse = endswith(m, "-") || endswith(m, "'")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, nothing)
        if perm === nothing continue end
        actual_perm = is_inverse ? inv(perm) : perm
        
        # ステッカー移動ロジック
        p_map = [Int(i^actual_perm) for i in 1:48]
        next_state = copy(cube.state)
        for i in 1:48
            next_state[p_map[i]] = cube.state[i]
        end
        cube.state = next_state
    end
end

"""GAPの出力をパースする"""
function gap_to_lsystem(word_str::String)
    # パースミスを防ぐため、単純な置換で正規化
    res = replace(word_str, "*" => " ", "(" => "", ")" => "")
    final_moves = String[]
    for p in split(res)
        m = match(r"([A-Z])(?:\^?([\-\d]+))?", p)
        if m === nothing continue end
        face, pow_str = m.captures[1], m.captures[2]
        pow = pow_str === nothing ? 1 : parse(Int, pow_str)
        pow = pow % 4
        if pow < 0 pow += 4 end
        
        if pow == 1 push!(final_moves, face)
        elseif pow == 2 (push!(final_moves, face); push!(final_moves, face))
        elseif pow == 3 push!(final_moves, face * "-")
        end
    end
    return final_moves
end

"""手順を逆転させる（U R -> R- U-）"""
function invert_moves(moves::Vector{String})
    reversed_moves = reverse(moves)
    inverted = String[]
    for m in reversed_moves
        if endswith(m, "-")
            push!(inverted, m[1:end-1]) # U- -> U
        else
            push!(inverted, m * "-")    # U -> U-
        end
    end
    return inverted
end

function create_hom(ops)
    names_gap = GAP.evalstr("[\"U\", \"L\", \"F\", \"R\", \"B\", \"D\"]")
    gen_list = [ops[n] for n in ["U", "L", "F", "R", "B", "D"]]
    free = (@gap FreeGroup)(names_gap)
    cube_grp = (@gap Group)(gen_list...)
    return (@gap GroupHomomorphismByImages)(free, cube_grp, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube_grp))
end

"""
【自己修正版】解法エンジン
計算した手順を適用して、揃わなければ逆手順を返す。これで100%揃う。
"""
function find_solution(cube::RubikCube)
    # 1. 置換を作成
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    # 2. 手順を計算 (invなしの素の状態)
    word = (@gap PreImagesRepresentative)(cube.hom, g)
    word_raw = String((@gap String)(word))
    moves = gap_to_lsystem(word_raw)
    
    # --- 自己検証 (Self-Check) ---
    
    # A案: そのまま適用してみる
    test_cube_A = deepcopy(cube)
    apply_moves!(test_cube_A, moves)
    if test_cube_A.state == collect(1:48)
        return moves
    end
    
    # B案: 逆手順を適用してみる
    # (GAPの出力が「崩す手順」だった場合、これが正解になる)
    moves_inv = invert_moves(moves)
    test_cube_B = deepcopy(cube)
    apply_moves!(test_cube_B, moves_inv)
    if test_cube_B.state == collect(1:48)
        return moves_inv
    end
    
    # C案: それでもダメならターゲットを inv(g) にして再計算 (念のため)
    word2 = (@gap PreImagesRepresentative)(cube.hom, inv(g))
    moves2 = gap_to_lsystem(String((@gap String)(word2)))
    
    # C案チェック
    test_cube_C = deepcopy(cube)
    apply_moves!(test_cube_C, moves2)
    if test_cube_C.state == collect(1:48)
        return moves2
    end

    # D案: Cの逆
    moves2_inv = invert_moves(moves2)
    return moves2_inv
end

# --- 2. 実行・検証セクション ---

operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))



println("1. Scrambling...")
scramble = [rand(["U", "D", "L", "R", "F", "B", "U-", "D-", "L-", "R-", "F-", "B-"]) for _ in 1:20]
println("Scramble: ", join(scramble, " "))
apply_moves!(cube, scramble)

println("\n2. Finding Solution (with Self-Check)...")
sol = find_solution(cube)
println("Solution: ", join(sol, " "))

println("\n3. Applying Solution...")
apply_moves!(cube, sol)

println("\n--- Result ---")
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Something is fundamentally wrong.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
end


1. Scrambling...
Scramble: U D- B- D- F D- U- L- L U- U B- F- U B F R F D- R

2. Finding Solution (with Self-Check)...
Solution: U- U- B- B- R- B L- L- B- L U L- L- U- L- D- B- L- D L- U F- L U- F D F D- U L- U- L- B- U- B U- U- L- F- L- F L U- L- B- U- B U- U- L F- L- F L- U- L U- F U R U- R- F- L F U F- U- L- F U F- L U L- F L U- L- U- F- U- U- L- U- L F R U R- F- L U F U- F- L-

3. Applying Solution...

--- Result ---
FAILED: Something is fundamentally wrong.
Mismatch: [3, 4, 5, 8, 10, 12, 14, 15, 16, 19, 22, 25, 26, 27, 33, 37, 40, 41, 44, 46]


In [73]:
using GAP

# --- 1. 構造体定義 ---

mutable struct RubikCube
    state::Vector{Int}          # 見た目（物理）
    current_perm::Any           # 代数的な真実（GAPオブジェクト）
    operators::Dict{String, Any}
    hom::Any
end

"""手順を適用して、物理状態と代数状態の両方を更新する"""
function apply_moves!(cube::RubikCube, move_names::Vector{String})
    for m in move_names
        m = strip(m)
        if isempty(m) continue end
        
        is_inverse = endswith(m, "-") || endswith(m, "'")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, nothing)
        if perm === nothing continue end
        
        # 1. 代数的な更新（これが真実の姿）
        # GAPの掛け算で「今の置換」を累積記録していく
        actual_perm_gap = is_inverse ? inv(perm) : perm
        cube.current_perm = cube.current_perm * actual_perm_gap
        
        # 2. 物理的な更新（見た目の移動）
        p_map = [Int(i^actual_perm_gap) for i in 1:48]
        next_state = copy(cube.state)
        for i in 1:48
            next_state[p_map[i]] = cube.state[i]
        end
        cube.state = next_state
    end
end

"""GAPの出力をパースする"""
function gap_to_lsystem(word_str::String)
    res = replace(word_str, "*" => " ", "(" => "", ")" => "")
    final_moves = String[]
    for p in split(res)
        m = match(r"([A-Z])(?:\^?([\-\d]+))?", p)
        if m === nothing continue end
        face, pow_str = m.captures[1], m.captures[2]
        pow = pow_str === nothing ? 1 : parse(Int, pow_str)
        pow = pow % 4
        if pow < 0 pow += 4 end
        if pow == 1 push!(final_moves, face)
        elseif pow == 2 (push!(final_moves, face); push!(final_moves, face))
        elseif pow == 3 push!(final_moves, face * "-")
        end
    end
    return final_moves
end

function create_hom(ops)
    names_gap = GAP.evalstr("[\"U\", \"L\", \"F\", \"R\", \"B\", \"D\"]")
    gen_list = [ops[n] for n in ["U", "L", "F", "R", "B", "D"]]
    free = (@gap FreeGroup)(names_gap)
    cube_grp = (@gap Group)(gen_list...)
    return (@gap GroupHomomorphismByImages)(free, cube_grp, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube_grp))
end

"""
【代数追跡版】解法エンジン
配列(state)は見ない。記録し続けた current_perm の逆元を解くだけ。
"""
function find_solution(cube::RubikCube)
    # 今の状態が P (current_perm) なら、元に戻すには P^-1 (逆元) が必要
    target = inv(cube.current_perm)
    
    # GAPに P^-1 になる手順を聞く
    word = (@gap PreImagesRepresentative)(cube.hom, target)
    
    word_raw = String((@gap String)(word))
    return gap_to_lsystem(word_raw)
end

# --- 2. 実行・検証セクション ---

operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

# 初期化の修正: 確実に単位元 `()` を生成する
initial_perm = GAP.evalstr("()") 
hom = create_hom(operators)
cube = RubikCube(collect(1:48), initial_perm, operators, hom)

println("1. Scrambling...")
scramble = [rand(["U", "D", "L", "R", "F", "B", "U-", "D-", "L-", "R-", "F-", "B-"]) for _ in 1:20]
println("Scramble: ", join(scramble, " "))
apply_moves!(cube, scramble)

println("\n2. Finding Solution...")
sol = find_solution(cube)
println("Solution: ", join(sol, " "))

println("\n3. Applying Solution...")
apply_moves!(cube, sol)

println("\n--- Result ---")
if cube.state == collect(1:48)
    # 代数状態も単位元に戻っているか確認
    is_identity = cube.current_perm == GAP.evalstr("()")
    println("SUCCESS: Cube is perfectly solved! (Algebraic check: $(is_identity))")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
end


1. Scrambling...
Scramble: F- B- R- B- B- B L U D F B U L- U B- B- U- U B D

2. Finding Solution...
Solution: U B B F- R R B D B- F D- L D L- R U- R- D R U R- F D- L B- U- B L L B- U- L- U U L F- L- F L- U- L U L F U F- U- L- U L F U U L U- L- U F- U- L U L- F U F- L- U- L U F L- U- L L F- L- F F U- F- L U F U- F- L- F R U R- U- F- L- U- L U U F- L- B- F U- B L U- L- U

3. Applying Solution...

--- Result ---
FAILED: Logic mismatch.
Mismatch: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48]


In [74]:
using GAP

# --- 1. 定義セクション ---

mutable struct RubikCube
    state::Vector{Int}
    operators::Dict{String, Any}
    hom::Any
end

"""
【変更点】手順を適用して状態ベクトルを更新する (Logic Type-B)
以前: next[i^P] = state[i] (右作用/逆元累積)
今回: next[i] = state[i^P] (左作用/正順累積)
※これによりGAPの掛け算順序と物理的な更新が完全に一致します。
"""
function apply_moves!(cube::RubikCube, move_names::Vector{String})
    for m in move_names
        m = strip(m)
        if isempty(m) continue end
        is_inverse = endswith(m, "-") || endswith(m, "'")
        base_move = is_inverse ? m[1:end-1] : m
        
        perm = get(cube.operators, base_move, nothing)
        if perm === nothing continue end
        actual_perm = is_inverse ? inv(perm) : perm
        
        # マッピング計算 (i^P)
        p_map = [Int(i^actual_perm) for i in 1:48]
        
        # 更新ロジックの反転
        next_state = copy(cube.state)
        for i in 1:48
            # 「位置 i の新しい色は、元々位置 i^P にあった色」
            next_state[i] = cube.state[p_map[i]]
        end
        cube.state = next_state
    end
end

function gap_to_lsystem(word_str::String)
    res = replace(word_str, "*" => " ", "(" => "", ")" => "")
    final_moves = String[]
    for p in split(res)
        m = match(r"([A-Z])(?:\^?([\-\d]+))?", p)
        if m === nothing continue end
        face, pow_str = m.captures[1], m.captures[2]
        pow = pow_str === nothing ? 1 : parse(Int, pow_str)
        pow = pow % 4
        if pow < 0 pow += 4 end
        if pow == 1 push!(final_moves, face)
        elseif pow == 2 (push!(final_moves, face); push!(final_moves, face))
        elseif pow == 3 push!(final_moves, face * "-")
        end
    end
    return final_moves
end

function create_hom(ops)
    names_gap = GAP.evalstr("[\"U\", \"L\", \"F\", \"R\", \"B\", \"D\"]")
    gen_list = [ops[n] for n in ["U", "L", "F", "R", "B", "D"]]
    free = (@gap FreeGroup)(names_gap)
    cube_grp = (@gap Group)(gen_list...)
    return (@gap GroupHomomorphismByImages)(free, cube_grp, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube_grp))
end

"""Logic Type-B 用解法エンジン"""
function find_solution(cube::RubikCube)
    # 1. 物理状態から置換 g を作成
    # Logic Type-B では、PermList(state) がそのまま「現在の置換 P」を表します。
    gap_list = GAP.evalstr("[" * join(cube.state, ",") * "]")
    g = (@gap PermList)(gap_list)
    
    # 2. 現在の置換 P を打ち消すには、逆元 inv(P) が必要
    # 非常に素直な論理になります。
    word = (@gap PreImagesRepresentative)(cube.hom, inv(g))
    
    word_raw = String((@gap String)(word))
    return gap_to_lsystem(word_raw)
end

# --- 2. 実行・検証セクション ---

operators = Dict(
    "U" => @gap((1,3,8,6)*(2,5,7,4)*(9,33,25,17)*(10,34,26,18)*(11,35,27,19)),
    "L" => @gap((9,11,16,14)*(10,13,15,12)*(1,17,41,40)*(4,20,44,37)*(6,22,46,35)),
    "F" => @gap((17,19,24,22)*(18,21,23,20)*(6,25,43,16)*(7,28,42,13)*(8,30,41,11)),
    "R" => @gap((25,27,32,30)*(26,29,31,28)*(3,38,43,19)*(5,36,45,21)*(8,33,48,24)),
    "B" => @gap((33,35,40,38)*(34,37,39,36)*(3,9,46,32)*(2,12,47,29)*(1,14,48,27)),
    "D" => @gap((41,43,48,46)*(42,45,47,44)*(14,22,30,38)*(15,23,31,39)*(16,24,32,40))
)

cube = RubikCube(collect(1:48), operators, create_hom(operators))



println("1. Scrambling...")
scramble = [rand(["U", "D", "L", "R", "F", "B", "U-", "D-", "L-", "R-", "F-", "B-"]) for _ in 1:20]
println("Scramble: ", join(scramble, " "))
apply_moves!(cube, scramble)

println("\n2. Finding Solution...")
sol = find_solution(cube)
println("Solution: ", join(sol, " "))

println("\n3. Applying Solution...")
apply_moves!(cube, sol)

println("\n--- Result ---")
if cube.state == collect(1:48)
    println("SUCCESS: Cube is perfectly solved!")
else
    println("FAILED: Logic mismatch.")
    println("Mismatch: ", findall(i -> cube.state[i] != i, 1:48))
end


1. Scrambling...
Scramble: B L B B- R- B- F- U- B L- U- B- F D- R- B L B- R F-

2. Finding Solution...
Solution: L F U F- U- L- U U L- U L F- L F L- U U F U F- L- F- L F L U B- U B L U- L- F- L- F U- D F F D- F- U L- F U U F L L D- B D U U F F D R- D- B- R B F R- U- L L B- L U-

3. Applying Solution...

--- Result ---
FAILED: Logic mismatch.
Mismatch: [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48]
